##  *Statistics coded:* Causes of death statistics

Prepared by  [**Katharina Koehler**](@kkatha) (EMOS).

Objective of this notebook is illustrating the "Statistics Explained" article [Causes of death statisitics](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Causes_of_death_statistics).

### Configuration

Put all libraries/packages/modules/etc... that are necessary so that you can run this notebook.

In [1]:
#install.packages(c("ggplot2", "plotly", "tidyr", "repr", "dplyr", "devtools", "restatapi"))
install.packages("restatapi")

package 'restatapi' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Katha\AppData\Local\Temp\RtmpchVSEA\downloaded_packages


In [2]:
library(ggplot2)
library(tidyr)
library(dplyr)
library(restatapi)
library(reshape2)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'restatapi' was built under R version 3.6.3"restatapi: - config file with the API version 1 loaded from GitHub (the 'current' API version number is 1).
           - 2 from the 4 cores are used for parallel computing.
           - 'auto' will be used for file download.
           - the Table of contents (TOC) was not pre-loaded into the deafult cache ('.restatapi_env').

Attaching package: 'reshape2'

The following object is masked from 'package:tidyr':

    smiths



### Overview/ Introduction

Creation of graphic: causes of death - standardised death rate, EU-28, 2016


#### First step:
Pulling data from Eurostat API "https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=hlth_cd_asdr2&lang=en"
id/ code used in function argument "hlth_cd_asdr2" to identify dataset

In [3]:
get_eurostat_toc(cache=TRUE,verbose=T)
search_eurostat_dsd(id="hlth_cd_asdr2",verbose=TRUE)

The TOC was cached in memory ('toc.xml.en' in '.restatapi_env').



,title,code,type,lastUpdate,lastModified,dataStart,dataEnd,values,unit,shortDescription,metadata.html,metadata.sdmx,downloadLink.tsv,downloadLink.sdmx
1,Consumers - monthly data,ei_bsco_m,dataset,2020.07.30,2020.07.30,1980M01,2020M07,272482,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsco_m.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsco_m.sdmx.zip
2,Consumers - quarterly data,ei_bsco_q,dataset,2020.07.30,2020.07.30,1990Q1,2020Q3,18540,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsco_q.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsco_q.sdmx.zip
3,Industry - monthly data,ei_bsin_m_r2,dataset,2020.07.30,2020.07.30,1980M01,2020M07,187603,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsin_m_r2.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsin_m_r2.sdmx.zip
4,Industry - quarterly data,ei_bsin_q_r2,dataset,2020.07.30,2020.07.30,1980Q1,2020Q3,92710,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsin_q_r2.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsin_q_r2.sdmx.zip
5,Construction - monthly data,ei_bsbu_m_r2,dataset,2020.07.30,2020.07.30,1980M01,2020M07,232094,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsbu_m_r2.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsbu_m_r2.sdmx.zip
6,Construction - quarterly data,ei_bsbu_q_r2,dataset,2020.07.30,2020.07.30,1981Q1,2020Q3,6327,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsbu_q_r2.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsbu_q_r2.sdmx.zip
7,Retail sale - monthly data,ei_bsrt_m_r2,dataset,2020.07.30,2020.07.30,1984M01,2020M07,116748,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsrt_m_r2.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bsrt_m_r2.sdmx.zip
8,Sentiment indicators - monthly data,ei_bssi_m_r2,dataset,2020.07.30,2020.07.30,1980M01,2020M07,113929,,,https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=metadata/ei_bcs_esms.sdmx.zip,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/ei_bssi_m_r2.tsv.gz,https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownlo

No DSD were provided.


NULL

In [4]:
dataset <- get_eurostat_data(id="hlth_cd_asdr2",label=TRUE,verbose=TRUE)
head(dataset)

0.8.6	id%3Dhlth_cd_asdr2	label%3DTRUE	verbose%3DTRUE
Trying to get from cache: b_hlth_cd_asdr2-2020.08.07-0-0
bulk url: http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/hlth_cd_asdr2.tsv.gz
class:data.tabledata.frame
The data was downloaded with bulk download.
bulk restat - nrow:814497;ncol:7;colnames:unit/sex/age/icd10/geo/time/values
cflags:FALSE
restat - nrow:814497;ncol:7;colnames:unit/sex/age/icd10/geo/time/values
The data was cached in memory ('b_hlth_cd_asdr2-2020.08.07-0-0' in '.restatapi_env').

restat - nrow:814497;ncol:7
Trying to download the DSD from: http://ec.europa.eu/eurostat/SDMX/diss-web/rest/datastructure/ESTAT/DSD_hlth_cd_asdr2
Warning by the download of the DSD file:
cannot open URL 'http://ec.europa.eu/eurostat/SDMX/diss-web/rest/datastructure/ESTAT/DSD_hlth_cd_asdr2': HTTP status was '500 Internal Server Error'
utils::download.file(dsd_endpoint, temp, get("dmethod", envir = .restatapi_env))
The dsd_xml is NULL. Please check i

unit,sex,age,icd10,geo,time,values
RT,F,TOTAL,A-R_V-Y,AT,2011,817.76
RT,F,TOTAL,A-R_V-Y,AT,2012,844.92
RT,F,TOTAL,A-R_V-Y,AT,2013,826.40
RT,F,TOTAL,A-R_V-Y,AT,2014,795.69
RT,F,TOTAL,A-R_V-Y,AT,2015,820.66
RT,F,TOTAL,A-R_V-Y,AT,2016,779.00


#### Second Step
Loading the complete dataset per bulk download via API (currently something with dsd from restatapi package doesn't work) -> Filtering the complete data set accordingly with R commands

In [ ]:
str(dataset)
levels(dataset$icd10)
attributes(dataset$icd10)$labels

### Main findings

The code here will help reproducing the material presented in the section [Main findings](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Causes_of_death_statistics).

In [ ]:
# your code

Don't forget to explain what you are doing...

### Additional material

At this stage, it is up to you to extend the work presented in the notebook with further analysis/visualisation you deem relevant. 